In [1]:
import numpy as np
from scipy.fftpack import fft2, ifft2
import time as tm
import plotly.graph_objects as go

In [2]:
# Parameters
g = 9.81  # Gravity (m/s^2)
L1 = 8814e3  # Distance Tokyo to LA (m)
L2 = 4688e3  # Distance Tokyo to New Guinea (m)
Nx = 256  # Number of grid points in x
Ny = 256  # Number of grid points in y
dx = L1 / (Nx - 1)  # Grid spacing in x
dy = L2 / (Ny - 1)  # Grid spacing in y
x = np.linspace(0, L1, Nx)
y = np.linspace(0, L2, Ny)
X, Y = np.meshgrid(x, y)
dt = 10  # Time step (s) - adjust for stability
hours = 3
t_end = 3600 * hours  # Simulation time
time = np.arange(0, t_end, dt)
Nt = len(time)

In [4]:
# Depth function (average 4km, sinusoidal variation)
def depth(x, y):
    return 4000 * (1+np.sin(2 * np.pi * (x-2*L1) / (5*L1)) * np.cos(2 * np.pi * y / L2))

D = depth(X, Y)

response = input("Would you like to plot the depth? (y/n): ")
if response == 'y':
    fig = go.Figure(data=[go.Surface(z=-D, x=x, y=y)])
    fig.update_layout(title='Depth (m)', autosize=False, width=800, height=800)
    fig.show()

In [5]:
# Initial conditions
x0 = L1 / 2
y0 = L2 / 2
sigma = L1 / 50

# Initial displacement (Gaussian pulse)
def initial_displacement(x, y, x0, y0, sigma):
    return 2*np.exp(-(((x - x0)/3)**2 / (2 * sigma**2) + ((y - y0))**2 / (2 * sigma**2))) + 3*np.exp(-((x - L1+1.5e6)**2 / (2 * sigma**2) + ((y -y0))**2 / (2 * sigma**2)))

eta = initial_displacement(X, Y, x0, y0, sigma)
eta_t = np.zeros(eta.shape)  # Initial velocity

response = input("Would you like to plot the initial displacement? (y/n): ")
if response == 'y':
    fig = go.Figure(data=[go.Surface(z=-eta, x=x, y=y)])
    fig.update_layout(title='Initial Displacement (m)', autosize=False, width=800, height=800)
    fig.show()

In [7]:
# Wavenumbers
kx = 2 * np.pi * np.fft.fftfreq(Nx, d=dx)
ky = 2 * np.pi * np.fft.fftfreq(Ny, d=dy)
KX, KY = np.meshgrid(kx, ky)

# Create a list of frames for the animation
def generate_frames():
    global eta, eta_t, Nt, dt, KX, KY
    for i in range(Nt):
        # Spatial derivatives in Fourier space
        eta_hat = fft2(eta)
        eta_xx_hat = -(KX**2) * eta_hat  # Second derivative in x
        eta_yy_hat = -(KY**2) * eta_hat  # Second derivative in y

        # Inverse FFT to get derivatives in spatial domain
        eta_x_x = np.real(ifft2(eta_xx_hat))
        eta_y_y = np.real(ifft2(eta_yy_hat))

        # RK4 stages
        k1_eta = eta_t
        k1_eta_t = g * D * (eta_x_x + eta_y_y)

        eta_temp = eta + 0.5 * dt * k1_eta
        
        eta_temp_hat = fft2(eta_temp)

        eta_x_x_temp_hat = -(KX**2) * eta_temp_hat
        eta_y_y_temp_hat = -(KY**2) * eta_temp_hat

        eta_x_x_temp = np.real(ifft2(eta_x_x_temp_hat))
        eta_y_y_temp = np.real(ifft2(eta_y_y_temp_hat))

        k2_eta = eta_t + 0.5 * dt * k1_eta_t
        k2_eta_t = g * D * (eta_x_x_temp + eta_y_y_temp)

        eta_temp = eta + 0.5 * dt * k2_eta

        eta_temp_hat = fft2(eta_temp)

        eta_x_x_temp_hat = -(KX**2) * eta_temp_hat
        eta_y_y_temp_hat = -(KY**2) * eta_temp_hat

        eta_x_x_temp = np.real(ifft2(eta_x_x_temp_hat))
        eta_y_y_temp = np.real(ifft2(eta_y_y_temp_hat))

        k3_eta = eta_t + 0.5 * dt * k2_eta_t
        k3_eta_t = g * D * (eta_x_x_temp + eta_y_y_temp)

        eta_temp = eta + dt * k3_eta

        eta_temp_hat = fft2(eta_temp)

        eta_x_x_temp_hat = -(KX**2) * eta_temp_hat
        eta_y_y_temp_hat = -(KY**2) * eta_temp_hat

        eta_x_x_temp = np.real(ifft2(eta_x_x_temp_hat))
        eta_y_y_temp = np.real(ifft2(eta_y_y_temp_hat))

        k4_eta = eta_t + dt * k3_eta_t
        k4_eta_t = g * D * (eta_x_x_temp + eta_y_y_temp)

        # Update eta and eta_t
        eta = eta + (dt / 6) * (k1_eta + 2 * k2_eta + 2 * k3_eta + k4_eta)
        eta_t = eta_t + (dt / 6) * (k1_eta_t + 2 * k2_eta_t + 2 * k3_eta_t + k4_eta_t)
        yield go.Frame(data=[go.Surface(z=eta, x=X, y=Y, colorscale='bluered')])
        

# Create the initial figure
fig = go.Figure(
    data=[go.Surface(z=np.sin(X) * np.cos(Y), x=X, y=Y, colorscale='bluered')],
    layout=go.Layout(
        title='3D Surface Plot Animation',
        scene=dict(
            xaxis_title='X (m)',
            yaxis_title='Y (m)',
            zaxis_title='Displacement',
            zaxis=dict(range=[-1, 1])
        ),
        updatemenus=[{
            'buttons': [
                {
                    'args': [None, {'frame': {'duration': 50, 'redraw': True}, 'fromcurrent': True}],
                    'label': 'Play',
                    'method': 'animate'
                },
                {
                    'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 0}}],
                    'label': 'Pause',
                    'method': 'animate'
                }
            ],
            'direction': 'left',
            'pad': {'r': 10, 't': 87},
            'showactive': False,
            'type': 'buttons',
            'x': 0.1,
            'xanchor': 'right',
            'y': 0,
            'yanchor': 'top'
        }],
        sliders=[{
            'steps': [
                {'args': [[str(i)], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 0}}],
                 'label': str(i),
                 'method': 'animate'} for i in range(Nt)
            ],
            'x': 0.1,
            'len': 0.9,
            'xanchor': 'left',
            'yanchor': 'top',
            'pad': {'b': 10, 't': 50},
            'currentvalue': {
                'font': {'size': 20},
                'prefix': 'Frame:',
                'visible': True,
                'xanchor': 'right'
            },
            'transition': {'duration': 0, 'easing': 'linear'}
        }]
    ),
    frames=list(generate_frames())
)

# Save the animation as an HTML file
fig.write_html(f"3d_surface_animation_with_slider_{tm.time()}.html")

# Show the plot
fig.show()


KeyboardInterrupt: 